In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF 
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


sc = SparkContext("local[*]")
spark = SparkSession(sc)


23/04/09 23:07:02 WARN Utils: Your hostname, Ashleys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.149 instead (on interface en0)
23/04/09 23:07:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 23:07:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#1. Clean the dataset

df = spark.read.csv( "/Users/ashleyrabanales/rabanales_scalable_data/yelp-dataset/yelp_review.csv", header=True)
df.printSchema()
df.show()
#df = df.withColumnRenamed("stars", "label")
df = df.withColumn("label", df["stars"].cast("double"))
#df = df.where(col("label").isNotNull())
df = df.dropna(subset=['label', 'text', 'funny', 'cool',"useful"])
df.show()
df.printSchema()

df = df.select('text', 'label')
df.show()

df = df.filter(df.label.isin(2.0,4.0))
df.show()

df = df.limit(1000)
df.show()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|  null| null|null|
|Staff was very he...|                   0|                   0|    0|      null|                null|  null| null|null|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6

+--------------------+-----+
|                text|label|
+--------------------+-----+
|This place is alw...|  4.0|
|Sometimes it feel...|  4.0|
|I have eaten at s...|  2.0|
|"Okay, so from wh...|  2.0|
|"While I really e...|  2.0|
|"Came here with m...|  4.0|
|"Our friends got ...|  4.0|
|A little on the p...|  4.0|
|"Went here with m...|  4.0|
|Been here twice n...|  4.0|
|We stopped in her...|  4.0|
|I often stop here...|  4.0|
|Not much here oth...|  4.0|
|I hiked this cany...|  4.0|
|Holy portion size...|  4.0|
|Flavor was actual...|  4.0|
|I don't do ice cr...|  4.0|
|I ate here with a...|  4.0|
|As a vegetarian, ...|  2.0|
|"The only downsid...|  4.0|
+--------------------+-----+
only showing top 20 rows



In [7]:

# Split the data into training and testing sets
(training_data, test_data) = df.randomSplit([0.7, 0.3], seed=1000)

# Define the ML pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="words")
TF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="tfFeatures")
idf = IDF(inputCol="tfFeatures", outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, TF, idf, lr])

# Train the model on the training data
model = pipeline.fit(training_data)

# Use the model to predict the labels of the first 10 comments in the test data
test_comments = test_data.select("text").limit(10)
predictions = model.transform(test_comments)

# Print the predictions
for row in predictions.collect():
    text, prediction = row["text"], row["prediction"]
    print(f"Comment: {text}\nPredicted rating: {prediction}\n")


23/04/09 23:12:21 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


23/04/09 23:12:22 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/09 23:12:22 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 23:12:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/09 23:12:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/04/09 23:12:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/09 23:12:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/04/09 23:12:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 23:12:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 23:12:24 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 23:12:25 WARN DAGScheduler: Broadcasting large task binary with size 4.

23/04/09 23:12:32 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
Comment: "Ah yes, Finish Line at West Towne Mall... I have to give this store a lower rating now because it used to be a nice size with lots of different styles of shoes. Now they down sized GREATLY and now they have an inferior selection. The staff, lazy... Everytime I have come in there looking for a pair of shoes I will have to stand by the cashiers desk just for them to notice that I need something. And even then they will give me a rhetorical question like 'did you need a size"" or they would be so enveloped in their cell phone that they wouldn't even notice I was there... I can't say much about this place except it was kept tidy looking."
Predicted rating: 2.0

Comment: "Came here again for a business dinner. We ordered the ""Daughter"" all in one menu (#3)
Predicted rating: 4.0

Comment: "Came here with my mother, husband and baby for brunch\/lunch. We were all quite happy with our choice. I th

In [3]:
#2 ML pipeline
#train data
(training_data, test_data) = df.randomSplit([0.7, 0.3], seed=1000)

tokenizer = Tokenizer(inputCol="text", outputCol="words")
TF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="tfFeatures")
idf = IDF(inputCol="tfFeatures", outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, TF, idf, lr])

# Train the model on the training data
model = pipeline.fit(training_data)

23/04/09 21:27:17 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


23/04/09 21:27:18 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/09 21:27:18 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:18 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/09 21:27:18 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/04/09 21:27:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/09 21:27:20 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/04/09 21:27:20 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:20 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:21 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:21 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:22 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:22 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:24 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/04/09 21:27:

In [8]:
#####################################
#3. Test data set, unlabeled text, your code here:
test_comments = test_data.select("text").limit(10)

#4. Make Prediction
prediction = model.transform(test_comments)

# Print the predictions
for row in prediction.collect():
    text, predictions = row["text"], row["prediction"]
    print(f"Comment: {text}\nPredicted rating: {predictions}\n")

#####################################

23/04/09 23:14:51 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
Comment: "Ah yes, Finish Line at West Towne Mall... I have to give this store a lower rating now because it used to be a nice size with lots of different styles of shoes. Now they down sized GREATLY and now they have an inferior selection. The staff, lazy... Everytime I have come in there looking for a pair of shoes I will have to stand by the cashiers desk just for them to notice that I need something. And even then they will give me a rhetorical question like 'did you need a size"" or they would be so enveloped in their cell phone that they wouldn't even notice I was there... I can't say much about this place except it was kept tidy looking."
Predicted rating: 2.0

Comment: "Came here again for a business dinner. We ordered the ""Daughter"" all in one menu (#3)
Predicted rating: 4.0

Comment: "Came here with my mother, husband and baby for brunch\/lunch. We were all quite happy with our choice. I th